In [1]:
# coding: utf-8

# In[1]:


# ref: https://www.kaggle.com/tilii7/tuned-logreg-oof-files
# https://www.kaggle.com/peterhurford/lightgbm-with-select-k-best-on-tfidf/code

import numpy as np
import pandas as pd
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from scipy.sparse import hstack
from sklearn.metrics import log_loss, matthews_corrcoef, roc_auc_score

class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('../input/train.csv').fillna(' ')
test = pd.read_csv('../input/test.csv').fillna(' ')
tr_ids = train[['id']]
train[class_names] = train[class_names].astype(np.int8)
target = train[class_names]

print(' Cleaning ...')
# PREPROCESSING PART
repl = {
    "yay!": " good ",
    "yay": " good ",
    "yaay": " good ",
    "yaaay": " good ",
    "yaaaay": " good ",
    "yaaaaay": " good ",
    ":/": " bad ",
    ":&gt;": " sad ",
    ":')": " sad ",
    ":-(": " frown ",
    ":(": " frown ",
    ":s": " frown ",
    ":-s": " frown ",
    "&lt;3": " heart ",
    ":d": " smile ",
    ":p": " smile ",
    ":dd": " smile ",
    "8)": " smile ",
    ":-)": " smile ",
    ":)": " smile ",
    ";)": " smile ",
    "(-:": " smile ",
    "(:": " smile ",
    ":/": " worry ",
    ":&gt;": " angry ",
    ":')": " sad ",
    ":-(": " sad ",
    ":(": " sad ",
    ":s": " sad ",
    ":-s": " sad ",
    r"\br\b": "are",
    r"\bu\b": "you",
    r"\bhaha\b": "ha",
    r"\bhahaha\b": "ha",
    r"\bdon't\b": "do not",
    r"\bdoesn't\b": "does not",
    r"\bdidn't\b": "did not",
    r"\bhasn't\b": "has not",
    r"\bhaven't\b": "have not",
    r"\bhadn't\b": "had not",
    r"\bwon't\b": "will not",
    r"\bwouldn't\b": "would not",
    r"\bcan't\b": "can not",
    r"\bcannot\b": "can not",
    r"\bi'm\b": "i am",
    "m": "am",
    "r": "are",
    "u": "you",
    "haha": "ha",
    "hahaha": "ha",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "hasn't": "has not",
    "haven't": "have not",
    "hadn't": "had not",
    "won't": "will not",
    "wouldn't": "would not",
    "can't": "can not",
    "cannot": "can not",
    "i'm": "i am",
    "m": "am",
    "i'll": "i will",
    "its": "it is",
    "it's": "it is",
    "'s": " is",
    "that's": "that is",
    "weren't": "were not",
}

keys = [i for i in repl.keys()]

new_train_data = []
new_test_data = []
ltr = train["comment_text"].tolist()
lte = test["comment_text"].tolist()
for i in ltr:
    arr = str(i).split()
    xx = ""
    for j in arr:
        j = str(j).lower()
        if j[:4] == 'http' or j[:3] == 'www':
            continue
        if j in keys:
            # print("inn")
            j = repl[j]
        xx += j + " "
    new_train_data.append(xx)
for i in lte:
    arr = str(i).split()
    xx = ""
    for j in arr:
        j = str(j).lower()
        if j[:4] == 'http' or j[:3] == 'www':
            continue
        if j in keys:
            # print("inn")
            j = repl[j]
        xx += j + " "
    new_test_data.append(xx)
train["new_comment_text"] = new_train_data
test["new_comment_text"] = new_test_data

trate = train["new_comment_text"].tolist()
tete = test["new_comment_text"].tolist()
for i, c in enumerate(trate):
    trate[i] = re.sub('[^a-zA-Z ?!]+', '', str(trate[i]).lower())
for i, c in enumerate(tete):
    tete[i] = re.sub('[^a-zA-Z ?!]+', '', tete[i])
train["comment_text"] = trate
test["comment_text"] = tete
del trate, tete
train.drop(["new_comment_text"], axis=1, inplace=True)
test.drop(["new_comment_text"], axis=1, inplace=True)

train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

# In[2]:


train.head()

# In[3]:


test.head()

# In[4]:


print(' Part 1/2 of vectorizing ...')
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 2),
    max_features=50000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

print(' Part 2/2 of vectorizing ...')
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(4, 6),
    max_features=50000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

train_features = hstack([train_char_features, train_word_features]).tocsr()
test_features = hstack([test_char_features, test_word_features]).tocsr()
print(train_features.shape, test_features.shape)

del all_text
del word_vectorizer
del char_vectorizer
del train_word_features
del train_char_features
del test_word_features
del test_char_features
import gc

gc.collect()

# In[5]:


target = target.values
print(target[:5])

# In[ ]:

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


 Cleaning ...
 Part 1/2 of vectorizing ...
 Part 2/2 of vectorizing ...
(159571, 100000) (153164, 100000)
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


In [6]:
from sklearn.feature_selection import SelectFromModel
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

model = LogisticRegression(solver='sag')
sfm = SelectFromModel(model, threshold=0.2)


# In[ ]:


def kf_train(k=5):
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=1001)
    train_pred, test_pred = np.zeros((159571, 6)), np.zeros((153164, 6))
    for j in range(6):
        train_x = sfm.fit_transform(train_features, target[:, j])
        print(train_x.shape)
        test_x = sfm.transform(test_features)
        fold_idx = 0
        for train_index, test_index in skf.split(train_x, target[:, j]):
            # data
            curr_x, curr_y = train_x[train_index], target[train_index][:, j]
            hold_out_x, hold_out_y = train_x[test_index], target[test_index][:, j]
            
            # fit
            rf = RandomForestClassifier(n_estimators=100,max_features='sqrt',min_samples_leaf=50,oob_score=True)
            rf.fit(curr_x,curr_y)

            hold_out_pred = rf.predict_proba(hold_out_x)
            print('hold out pred done')
            curr_train_pred = rf.predict_proba(curr_x)
            print('curr train pred done')
            # floating point exception (core dumped), seems updated lightgbm fix this
            print(fold_idx,log_loss(hold_out_y,hold_out_pred),log_loss(curr_y,curr_train_pred))
            print(roc_auc_score(hold_out_y,hold_out_pred[:,1]),roc_auc_score(curr_y,curr_train_pred[:,1]))
            fold_idx += 1

            train_pred[test_index, j] = list(hold_out_pred[:,1].flatten())
            y_test = rf.predict_proba(test_x)[:,1]
            test_pred[:, j] += y_test
        print('=========', class_names[j])
    test_pred = test_pred / k
    return train_pred, test_pred


train_pred, test_pred = kf_train(5)

# In[ ]:


import pickle

with open('../features/rf1_feat.pkl', 'wb') as fout:
    pickle.dump([train_pred, test_pred], fout)
print(test_pred[:5])

(159571, 24291)
hold out pred done
curr train pred done
0 0.15312754867745565 0.14360858348565422
0.9563407386911488 0.9709035305151267
hold out pred done
curr train pred done
1 0.15004052319536879 0.14523542916440946
0.9589828896864575 0.9702390123933107
hold out pred done
curr train pred done
2 0.15374012637261034 0.14347498584438698
0.9553292326519705 0.9709881006863271
hold out pred done
curr train pred done
3 0.15111452548780982 0.14498623987595957
0.9588783894220568 0.970472740895429
hold out pred done
curr train pred done
4 0.15318092471758915 0.1444482876872729
0.9565960199500014 0.970434930395617
========= toxic
(159571, 6358)
hold out pred done
curr train pred done
0 0.026200849562584182 0.024393950920948357
0.9834905295341142 0.9892345248271001
hold out pred done
curr train pred done
1 0.02485302796306221 0.024739270167841474
0.9874761442452752 0.988636087688174
hold out pred done
curr train pred done
2 0.025724744345377654 0.02449048109690013
0.9854929230201397 0.9890390278

In [7]:
%%time 

from sklearn.ensemble import GradientBoostingClassifier

model = LogisticRegression(solver='sag')
sfm = SelectFromModel(model, threshold=0.2)


# In[ ]:


def kf_train(k=5):
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=1001)
    train_pred, test_pred = np.zeros((159571, 6)), np.zeros((153164, 6))
    for j in range(6):
        train_x = sfm.fit_transform(train_features, target[:, j])
        print(train_x.shape)
        test_x = sfm.transform(test_features)
        fold_idx = 0
        for train_index, test_index in skf.split(train_x, target[:, j]):
            # data
            curr_x, curr_y = train_x[train_index], target[train_index][:, j]
            hold_out_x, hold_out_y = train_x[test_index], target[test_index][:, j]
            
            # fit
            rf = GradientBoostingClassifier(n_estimators=100,max_depth=3,
                                            max_features='sqrt',min_samples_leaf=50)
            rf.fit(curr_x,curr_y)

            hold_out_pred = rf.predict_proba(hold_out_x)
            print('hold out pred done')
            curr_train_pred = rf.predict_proba(curr_x)
            print('curr train pred done')
            # floating point exception (core dumped), seems updated lightgbm fix this
            print(fold_idx,log_loss(hold_out_y,hold_out_pred),log_loss(curr_y,curr_train_pred))
            print(roc_auc_score(hold_out_y,hold_out_pred[:,1]),roc_auc_score(curr_y,curr_train_pred[:,1]))
            fold_idx += 1

            train_pred[test_index, j] = list(hold_out_pred[:,1].flatten())
            y_test = rf.predict_proba(test_x)[:,1]
            test_pred[:, j] += y_test
        print('=========', class_names[j])
    test_pred = test_pred / k
    return train_pred, test_pred


train_pred, test_pred = kf_train(5)

# In[ ]:


import pickle

with open('../features/gbrt1_feat.pkl', 'wb') as fout:
    pickle.dump([train_pred, test_pred], fout)
print(test_pred[:5])

(159571, 24295)
hold out pred done
curr train pred done
0 0.17324130565629228 0.1658377861998216
0.931159014340736 0.9400877161244077
hold out pred done
curr train pred done
1 0.1677725737488159 0.16789619540300962
0.9403687272477792 0.9414593207184202
hold out pred done
curr train pred done
2 0.17126010412453882 0.1673507752472627
0.9336669822039145 0.9399781227454979
hold out pred done
curr train pred done
3 0.16690990296452432 0.16550322173158946
0.9379952087658139 0.9401682312634017
hold out pred done
curr train pred done
4 0.16993342426688393 0.16630487836107238
0.9332119540894748 0.9391352110927718
========= toxic
(159571, 6358)
hold out pred done
curr train pred done
0 0.03168559597384564 0.026145637132560275
0.9731305022142797 0.9834744979687572
hold out pred done
curr train pred done
1 0.028033578671337416 0.026940780404558093
0.9823432936742005 0.9847302146946367
hold out pred done
curr train pred done
2 0.031085402579847585 0.02657438834208363
0.9778550135655963 0.9846745351